# Experiment 
## Part1: prepare model and data

In [1]:
import time
import torch
import librosa
import numpy as np
import torch.nn as nn
from sklearn import preprocessing
np.random.seed(42)
# step1: define the entire ia-net-lite model

class GLayerNorm(nn.Module):
    """Global Layer Normalization for TasNet."""

    def __init__(self, channels, eps=1e-5):
        super(GLayerNorm, self).__init__()
        self.eps = eps
        self.norm_dim = channels
        self.gamma = nn.Parameter(torch.Tensor(channels))
        self.beta = nn.Parameter(torch.Tensor(channels))
        # self.register_parameter('weight', self.gamma)
        # self.register_parameter('bias', self.beta)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)

    def forward(self, sample):
        """Forward function.
        Args:
            sample: [batch_size, channels, length]
        """
        if sample.dim() != 3:
            raise RuntimeError('{} only accept 3-D tensor as input'.format(
                self.__name__))
        # [N, C, T] -> [N, T, C]
        sample = torch.transpose(sample, 1, 2)
        # Mean and variance [N, 1, 1]
        mean = torch.mean(sample, (1, 2), keepdim=True)
        var = torch.mean((sample - mean) ** 2, (1, 2), keepdim=True)
        sample = (sample - mean) / torch.sqrt(var + self.eps) * \
                 self.gamma + self.beta
        # [N, T, C] -> [N, C, T]
        sample = torch.transpose(sample, 1, 2)
        return sample



class Bottleneck(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=7, stride=1, downsample=None, expansion=1):
        super(Bottleneck, self).__init__()
        inplanes_ = inplanes * expansion
        pad = (kernel_size - 1) // 2
        self.conv1 = nn.Conv1d(inplanes, inplanes_, kernel_size=1, bias=False)
        self.bn1 = GLayerNorm(inplanes_)
        self.conv2 = nn.Conv1d(inplanes_, inplanes_, kernel_size=kernel_size, stride=stride,
                               padding=pad, bias=False, groups=inplanes_)
        self.bn2 = GLayerNorm(inplanes_)
        self.conv3 = nn.Conv1d(inplanes_, planes, kernel_size=1, bias=1)
        self.bn3 = GLayerNorm(planes)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ConvMulti(nn.Module):
    def __init__(self, inplanes, planes, stride=4):
        super(ConvMulti, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv_7 = nn.Conv1d(inplanes, planes//2, kernel_size=7, stride=stride,
                                padding=3, bias=False, groups=planes//2)
        self.bn_7 = GLayerNorm(planes//2)

        self.conv_5 = nn.Conv1d(inplanes, planes//2, kernel_size=5, stride=stride,
                                padding=2, bias=False, groups=planes//2)
        self.bn_5 = GLayerNorm(planes//2)

    def forward(self, x):
        output_7 = self.bn_7(self.conv_7(x))
        output_5 = self.bn_5(self.conv_5(x))
        return self.relu(torch.cat([output_7, output_5], dim=1))


class BottleneckMulti(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=7, stride=1, downsample=None, expansion=1):
        super(BottleneckMulti, self).__init__()
        inplanes_ = inplanes * expansion
        pad = (kernel_size - 1) // 2
        self.conv1 = nn.Conv1d(inplanes, inplanes_, kernel_size=1, bias=False)
        self.bn1 = GLayerNorm(inplanes_)
        # self.conv2 = nn.Conv1d(inplanes_, inplanes_, kernel_size=kernel_size, stride=stride,
        #                        padding=pad, bias=False, groups=inplanes_)
        # self.bn2 = GLayerNorm(inplanes_)
        self.conv2 = ConvMulti(inplanes_, inplanes_, stride)
        self.conv3 = nn.Conv1d(inplanes_, planes, kernel_size=1, bias=1)
        self.bn3 = GLayerNorm(planes)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        # out = self.bn2(out)
        # out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class Encoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Encoder, self).__init__()
        self.conv_7 = nn.Conv1d(in_channels, out_channels//2, kernel_size=7, stride=4, padding=3, bias=False)
        self.conv_5 = nn.Conv1d(in_channels, out_channels//2, kernel_size=5, stride=4, padding=2, bias=False)

    def forward(self, x):
        output_7 = self.conv_7(x)
        output_5 = self.conv_5(x)
        return torch.cat([output_7, output_5], dim=1)


class MobileNetV2(nn.Module):
    def __init__(self, block, layers, n_spk=4, num_emed=128, mode='train'):
        super(MobileNetV2, self).__init__()
        self.mode = mode
        self.inplanes = 32
        self.n_spk = n_spk
        self.num_emed = num_emed
        # self.conv1 = nn.Conv1d(1, 32, kernel_size=7, stride=4, padding=3, bias=False)
        self.conv1 = Encoder(1, 32)
        self.bn1 = GLayerNorm(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, layers[0], stride=1, expansion=1)
        self.layer2 = self._make_layer(block, 24, layers[1], stride=4, expansion=6)
        self.layer3 = self._make_layer(block, 32, layers[2], stride=4, expansion=6)
        self.layer4 = self._make_layer(block, 64, layers[3], stride=4, expansion=6)
        self.layer5 = self._make_layer(block, 96, layers[4], stride=1, expansion=6)
        self.layer6 = self._make_layer(block, 160, layers[5], stride=4, expansion=6)
        self.layer7 = self._make_layer(block, 320, layers[6], stride=1, expansion=6)
        self.conv8 = nn.Conv1d(320, 1280, kernel_size=1, stride=1, bias=False)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(1280, num_emed*n_spk)

    def _make_layer(self, block, planes, blocks, stride, expansion):
        downsample = nn.Sequential(
            nn.Conv1d(self.inplanes, planes,
                      kernel_size=1, stride=stride, bias=False),
            GLayerNorm(planes),
        )

        layers = []
        layers.append(block(self.inplanes, planes, stride=stride, downsample=downsample, expansion=expansion))
        self.inplanes = planes
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, expansion=expansion))

        return nn.Sequential(*layers)

    def forward_once(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)

        x = self.conv8(x)
        x = self.avgpool(x).squeeze(-1)
        x = self.fc(x)
        features =[]
        start = 0
        for i in range(self.n_spk):
            f = x[:, start:start+self.num_emed]
            start += self.num_emed
            features.append(f)
        return features

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)


def mobilenet_19(**kwargs):
    """Constructs a MobileNetV2-19 model.
    """
    n_spk = kwargs["n_spk"] if "n_spk" in kwargs else 4
    num_emed = kwargs["num_emed"] if "num_emed" in kwargs else 128
    mode = kwargs["mode"] if "mode" in kwargs else 'train'
    model = MobileNetV2(BottleneckMulti, [1, 2, 3, 4, 3, 3, 1], n_spk, num_emed, mode)
    return model

In [2]:
# define the distributed ia-net-lite
class Part_1(nn.Module):
    def __init__(self, backbone, mode="train"):
        super(Part_1, self).__init__()
        self.mode = mode
        self.conv1 = backbone.conv1
        self.bn1 = backbone.bn1
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4

    def forward_once(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)


class Part_2(nn.Module):
    def __init__(self, backbone, mode='train'):
        super(Part_2, self).__init__()
        self.mode = mode
        self.layer5 = backbone.layer5
        self.layer6 = backbone.layer6

    def forward_once(self, x):
        x = self.layer5(x)
        x = self.layer6(x)
        return x

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)


class Part_3(nn.Module):
    def __init__(self, backbone, mode='train'):
        super(Part_3, self).__init__()
        self.num_emed = backbone.num_emed
        self.n_spk = backbone.n_spk
        self.mode = mode
        self.layer7 = backbone.layer7
        self.conv8 = backbone.conv8
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = backbone.fc

    def forward_once(self, x):
        x = self.layer7(x)
        x = self.conv8(x)
        x = self.avgpool(x).squeeze(-1)
        x = self.fc(x)
        features = []
        start = 0
        for i in range(self.n_spk):
            f = x[:, start:start + self.num_emed]
            start += self.num_emed
            features.append(f)
        return features

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)


def build_part(backbone, mode='train'):
    """Constructs a MobileNetV2-19 model.
    """
    # backbone = mobilenet_19()
    part1 = Part_1(backbone, mode=mode)
    part2 = Part_2(backbone, mode=mode)
    part3 = Part_3(backbone, mode=mode)
    return part1, part2, part3

In [3]:
# create the entire model and split model
model = mobilenet_19(num_emed=256, n_spk=4, mode='inference')
model_path = "./model_params.pkl"
model.load_state_dict(torch.load(model_path))
model = model.cuda()
model.eval()
model_part1, model_part2, model_part3 = build_part(model, mode="inference")
model_part1.eval()
model_part2.eval()
model_part3.eval()

Part_3(
  (layer7): Sequential(
    (0): BottleneckMulti(
      (conv1): Conv1d(160, 960, kernel_size=(1,), stride=(1,), bias=False)
      (bn1): GLayerNorm()
      (conv2): ConvMulti(
        (relu): ReLU(inplace=True)
        (conv_7): Conv1d(960, 480, kernel_size=(7,), stride=(1,), padding=(3,), groups=480, bias=False)
        (bn_7): GLayerNorm()
        (conv_5): Conv1d(960, 480, kernel_size=(5,), stride=(1,), padding=(2,), groups=480, bias=False)
        (bn_5): GLayerNorm()
      )
      (conv3): Conv1d(960, 320, kernel_size=(1,), stride=(1,))
      (bn3): GLayerNorm()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv1d(160, 320, kernel_size=(1,), stride=(1,), bias=False)
        (1): GLayerNorm()
      )
    )
  )
  (conv8): Conv1d(320, 1280, kernel_size=(1,), stride=(1,), bias=False)
  (avgpool): AdaptiveAvgPool1d(output_size=1)
  (fc): Linear(in_features=1280, out_features=1024, bias=True)
)

In [8]:
# prepare data
def load_data(path_list):
    scaler = preprocessing.MaxAbsScaler()
    sources = []
    mix = np.zeros(160000, dtype=np.float32)
    for path in path_list:
        data, _ = librosa.load(path, sr=16000, mono=True)
        mix += data
        data = scaler.fit_transform(data.reshape(-1, 1)).T
        sources.append(data)
    mix = scaler.fit_transform(mix.reshape(-1, 1)).T
    return mix
    
s1_normal = "./data/normals/pump/0.wav"
s2_normal = "./data/normals/slider/0.wav"
s3_normal = "./data/normals/fan/0.wav"
s4_normal = "./data/normals/valve/0.wav"
s1_abnormal = "./data/abnormals/pump/0.wav"
s2_abnormal = "./data/abnormals/slider/0.wav"
s3_abnormal = "./data/abnormals/fan/0.wav"
s4_abnormal = "./data/abnormals/valve/0.wav"

mix_baseline = load_data([s1_normal, s2_normal, s3_normal, s4_normal])
mix = load_data([s1_abnormal, s2_normal, s3_normal, s4_abnormal])
mix_baseline_short = mix_baseline[:, :98304]
mix_short = mix[:, :98304]
print(mix_baseline.shape)
print(mix.shape)
print(mix_baseline_short.shape)
print(mix_short.shape)
# conver numpy to Tensor
mix_baseline_short = torch.Tensor(mix_baseline_short).to("cuda:0")
mix_short = torch.Tensor(mix_short).to("cuda:0")

(1, 160000)
(1, 160000)
(1, 98304)
(1, 98304)


In [10]:
# compute results of entire ia-net-lite
print(mix_baseline_short.size())
print(mix_short.size())
t_start = time.perf_counter()
features_baseline = model(mix_baseline_short.unsqueeze(0))
t_entire = time.perf_counter() - t_start
t_start = time.perf_counter()
features = model(mix_short.unsqueeze(0))
t_entire = time.perf_counter() - t_start
print("Entire computation time is {}".format(t_entire))

torch.Size([1, 98304])
torch.Size([1, 98304])
Entire computation time is 0.07651120000082301


In [11]:
# compute results of distributed ia-net-lite
t_start = time.perf_counter()
features_dis = model_part1(mix_short.unsqueeze(0))
features_dis = model_part2(features_dis)
features_dis = model_part3(features_dis)
t_dis = time.perf_counter() - t_start
print("Dis. computation time is {}".format(t_dis))

Dis. computation time is 0.08116300000074261


In [12]:
def calculate_anomal_score(feature_baseline, feature, n=4):
    ans = []
    for i in range(n):
        d = (feature_baseline[i] - feature[i]).pow(2).sum(1)
        ans.append(d.item())
    return ans

## Part2: How to split data 
To split the data, the FC needs to separate from the other layers

conv layers - output - fc-layer - result
a, b, c           outa+outb+outc  - output - fc-layer - result


input --- 32 inputs -> vnf1 - vnf2 - vnf3 - server

In [13]:
class ConvLayers(nn.Module):
    def __init__(self, backbone, mode="train"):
        super(ConvLayers, self).__init__()
        self.mode = mode
        self.conv1 = backbone.conv1
        self.bn1 = backbone.bn1
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
        self.layer5 = backbone.layer5
        self.layer6 = backbone.layer6
        self.layer7 = backbone.layer7
        self.conv8 = backbone.conv8


    def forward_once(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.layer6(x)
        x = self.layer7(x)
        x = self.conv8(x)
        return x

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)

class FCLayers(nn.Module):
    def __init__(self, backbone, mode='train') -> None:
        super(FCLayers, self).__init__()
        self.num_emed = backbone.num_emed
        self.n_spk = backbone.n_spk
        self.mode = mode
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = backbone.fc
    
    def forward_once(self, x):
        x = self.avgpool(x).squeeze(-1)
        x = self.fc(x)
        features = []
        start = 0
        for i in range(self.n_spk):
            f = x[:, start:start + self.num_emed]
            start += self.num_emed
            features.append(f)
        return features

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)

conv_layers = ConvLayers(model, mode="inference")
fc_layer = FCLayers(model, mode="inference")

In [21]:
## splite the 2s segment to 2 parts along the time domain
import math
def splite_data(data, n, dim):
    t = data.size()[-1]
    l = t // n
    data_list = torch.split(data, l, dim=dim)
    return data_list

class Spliter:
    def __init__(self, model, input_size, s, n) -> None:
        '''
        model:
        input_size: shape of input data, size should be like [b, c, t] 
        s: stride size, depends on the model
        n: numbero of splited data
        '''
        assert n <= input_size // s
        self.model = model
        self.s = s
        self.n = n
        sub_size = input_size // n
        self.size = sub_size

    def compute_once(self, x):
        b, c, t = x.shape
        left = t % self.s
        if left != 0:
            zeros_pad = torch.zeros([b, c, self.s - left], dtype=torch.float32).cuda()
            x = torch.cat([x, zeros_pad], dim=2)
        return self.model(x)

    def split_and_compute(self, x):
        if self.n == 1:
            return self.model(x)
        if len(x.size()) == 2:
            x = x.unsqueeze(0)
        split_list = torch.split(x, self.size, dim=2)
        ans = []
        for sub_input in split_list:
            out = self.compute_once(sub_input)
            '''
            此处可以添加发送的逻辑
            '''
            ans.append(out)
        return ans

s = Spliter(conv_layers, 98304, 1024, 32)
t_start = time.perf_counter()
ans = s.split_and_compute(mix_short)
print(ans[0].size())
t_split = time.perf_counter() - t_start
feature_combine = torch.cat(ans, dim=2)
feature_combine = fc_layer(feature_combine)
print("Split computation time: {}".format(t_split))

torch.Size([1, 1280, 3])
Split computation time: 0.8042284999992262


## Part3 split & or not
Let's only split & send into first two parts
First we define a combiner
32 - idx0, idx1 -> part2 

In [22]:
class Combiner:
    def __init__(self, model, s_prev, s_cur, n = None) -> None:
        self.model = model
        self.cache = []
        self.n = math.floor(s_prev/s_cur) if n is None else n

    def combine_and_compute(self, x):
        self.cache.append(x)
        if len(self.cache) < self.n:
            return None
        input = torch.cat(self.cache, dim=2)
        self.cache = []
        return self.model(input)

In [23]:
s1 = Spliter(model_part1, 98304, 256, 32)
ans = s1.split_and_compute(mix_short)
print(ans[0].size())
s2 = Spliter(model_part2, 12, 4, 1)
ans_ = []
for x in ans:
    out = s2.split_and_compute(x)
    ans_.append(out)
feature_combine_1 = torch.cat(ans_, dim=2)
print(feature_combine_1.size())
feature_combine_1 = model_part3(feature_combine_1)

torch.Size([1, 64, 12])
torch.Size([1, 160, 96])


In [24]:
# Compare distance
distance_entire = calculate_anomal_score(features_baseline, features)
distance_distributed = calculate_anomal_score(features_baseline, features_dis)
distance_split = calculate_anomal_score(features_baseline, feature_combine)
distance_split_1 = calculate_anomal_score(features_baseline, feature_combine_1)
print(distance_entire)
print(distance_distributed)
print(distance_split)
print(distance_split_1)


[11.72973918914795, 0.1781705617904663, 0.03114146739244461, 2.6397199630737305]
[11.72973918914795, 0.1781705617904663, 0.03114146739244461, 2.6397199630737305]
[8.744229316711426, 1.8077257871627808, 2.1615681648254395, 54.81706237792969]
[7.211784362792969, 12.4138765335083, 0.17199158668518066, 56.764930725097656]


In [25]:
class Part_Conv_3(nn.Module):
    def __init__(self, backbone, mode='train'):
        super(Part_Conv_3, self).__init__()
        self.num_emed = backbone.num_emed
        self.n_spk = backbone.n_spk
        self.mode = mode
        self.layer7 = backbone.layer7
        self.conv8 = backbone.conv8

    def forward_once(self, x):
        x = self.layer7(x)
        x = self.conv8(x)
        return x

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)

class Part_FC_3(nn.Module):
    def __init__(self, backbone, mode='train'):
        super(Part_FC_3, self).__init__()
        self.num_emed = backbone.num_emed
        self.n_spk = backbone.n_spk
        self.mode = mode
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = backbone.fc

    def forward_once(self, x):
        x = self.avgpool(x).squeeze(-1)
        x = self.fc(x)
        features = []
        start = 0
        for i in range(self.n_spk):
            f = x[:, start:start + self.num_emed]
            start += self.num_emed
            features.append(f)
        return features

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)

part_3_conv = Part_Conv_3(model, mode="inference")
part_3_fc = Part_FC_3(model, mode="inference")

In [28]:
s1 = Spliter(model_part1, 98304, 256, 32)
ans = s1.split_and_compute(mix_short)
print(len(ans))
c2 = Combiner(model_part2, -1, -1, 2)
ans_2 = []
count = 0
for x in ans:
    out = c2.combine_and_compute(x)
    if out is not None:
        count += 1
        ans_2.append(out)
print(len(ans_2))
c3 = Combiner(part_3_conv, -1, -1, 8)
ans_3 = []
for x in ans_2:
    out = c3.combine_and_compute(x)
    if out is not None:
        ans_3.append(out)
print(ans_3[0].size())
print(len(ans_3))
feature_combine_2 = torch.cat(ans_3, dim=2)
print(feature_combine_2.size())
feature_combine_2 = part_3_fc(feature_combine_2)

32
16
torch.Size([1, 1280, 48])
2
torch.Size([1, 1280, 96])


In [29]:
# Compare distance
distance_entire = calculate_anomal_score(features_baseline, features)
distance_distributed = calculate_anomal_score(features_baseline, features_dis)
distance_split = calculate_anomal_score(features_baseline, feature_combine)
distance_split_1 = calculate_anomal_score(features_baseline, feature_combine_1)
distance_split_2 = calculate_anomal_score(features_baseline, feature_combine_2)
print(distance_entire)
print(distance_distributed)
print(distance_split)
print(distance_split_1)
print(distance_split_2)

[11.72973918914795, 0.1781705617904663, 0.03114146739244461, 2.6397199630737305]
[11.72973918914795, 0.1781705617904663, 0.03114146739244461, 2.6397199630737305]
[8.744229316711426, 1.8077257871627808, 2.1615681648254395, 54.81706237792969]
[7.211784362792969, 12.4138765335083, 0.17199158668518066, 56.764930725097656]
[8.683256149291992, 0.027838263660669327, 1.0062261819839478, 17.375463485717773]


## 如果不分割fc层，直接对最后的结果处理，sum or mean

In [237]:
class Part_3_mod(nn.Module):
    def __init__(self, backbone, mode='train'):
        super(Part_3_mod, self).__init__()
        self.num_emed = backbone.num_emed
        self.n_spk = backbone.n_spk
        self.mode = mode
        self.layer7 = backbone.layer7
        self.conv8 = backbone.conv8
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = backbone.fc

    def forward_once(self, x):
        x = self.layer7(x)
        x = self.conv8(x)
        x = self.avgpool(x).squeeze(-1)
        x = self.fc(x)
        return x

    def forward(self, input):
        if self.mode == "train":
            features = self.forward_once(input[0])
            features_ = self.forward_once(input[1])
            return features, features_
        else:
            return self.forward_once(input)
model_part3_mod = Part_3_mod(model, mode="inference")

def align_feature(x, n_spk=4, num_emed=256):
    features = []
    start = 0
    for i in range(n_spk):
        f = x[:, start:start + num_emed]
        start += num_emed
        features.append(f)
    return features

# spliter -> vnf1 -----------  Combiner2 - vnf2 ---------Combiner3 -vnf3-convlayer - fc-layer
# (n1 + n2 + n3) chunk_gap + compute+time
#  -------------
#  -------------
#  -------------

In [238]:
with torch.no_grad():
    s1 = Spliter(model_part1, 98304, 256, 32)
    ans = s1.split_and_compute(mix_short)
    print(ans[0].size())
    s2 = Spliter(model_part2, 12, 4, 1)
    ans_ = []
    features_new = torch.zeros([1, 1024], dtype=torch.float64).cuda()
    for x in ans:
        out = s2.split_and_compute(x)
        out = model_part3_mod(out)
        features_new += out
    features_new = align_feature(features_new)


    

torch.Size([1, 64, 12])
